In [1]:
import numpy as np
from squlearn.encoding_circuit import ChebyshevPQC,QCNNEncodingCircuit
from squlearn.observables import SummedPaulis, CustomObservable

pqc = ChebyshevPQC(2, 2, 2)
obs = SummedPaulis(2)


np.random.seed(42)
circuit1 = pqc.get_circuit(np.random.rand(pqc.num_features),np.random.rand(pqc.num_parameters))
circuit2 = pqc.get_circuit(np.random.rand(pqc.num_features),np.random.rand(pqc.num_parameters))
observable1 = obs.get_operator(np.random.rand(obs.num_parameters))
observable2 = obs.get_operator(np.random.rand(obs.num_parameters))

In [2]:
circuit1

In [3]:
observable2

SparsePauliOp(['II', 'IZ', 'ZI'],
              coeffs=[0.51423444+0.j, 0.59241457+0.j, 0.04645041+0.j])

In [4]:
from squlearn.util.optree import OpTree,OpTreeList

measured_tree = OpTree.gen_expectation_tree(OpTreeList([circuit1,circuit2]), OpTreeList([observable1,observable2]))

print(measured_tree)

[1.0*[1.0*
     ┌─────────────┐┌─────────────┐                 ┌────────────┐»
q_0: ┤ Ry(0.73199) ├┤ Rx(0.18518) ├───────■─────────┤ Rx(1.0281) ├»
     ├─────────────┤├─────────────┤┌──────┴───────┐┌┴────────────┤»
q_1: ┤ Ry(0.59866) ├┤ Rx(0.04918) ├┤ Rz(0.058084) ├┤ Rx(0.18951) ├»
     └─────────────┘└─────────────┘└──────────────┘└─────────────┘»
«                    ┌──────────────┐
«q_0: ───────■───────┤ Ry(0.020584) ├
«     ┌──────┴──────┐├─────────────┬┘
«q_1: ┤ Rz(0.70807) ├┤ Ry(0.96991) ├─
«     └─────────────┘└─────────────┘ 

 with observable 
SparsePauliOp(['II', 'IZ', 'ZI'],
              coeffs=[0.45606998+0.j, 0.78517596+0.j, 0.19967378+0.j])
, 1.0*
     ┌─────────────┐┌─────────────┐                 ┌────────────┐»
q_0: ┤ Ry(0.73199) ├┤ Rx(0.18518) ├───────■─────────┤ Rx(1.0281) ├»
     ├─────────────┤├─────────────┤┌──────┴───────┐┌┴────────────┤»
q_1: ┤ Ry(0.59866) ├┤ Rx(0.04918) ├┤ Rz(0.058084) ├┤ Rx(0.18951) ├»
     └─────────────┘└─────────────┘└──────────────┘└────

In [5]:
from qiskit.primitives import Sampler

OpTree.evaluate.evaluate_tree_with_sampler(measured_tree,{},Sampler())

array([[0.64149003, 0.65482746],
       [1.10656431, 1.00928855]])

In [6]:

from qiskit.primitives import Estimator

OpTree.evaluate.evaluate_tree_with_estimator(measured_tree,{},Estimator())

array([[0.64149003, 0.65482746],
       [1.10656431, 1.00928855]])

In [13]:
qcnn = QCNNEncodingCircuit(4)
qcnn.convolution()
qcnn.pooling(measurement=True)
qcnn.fully_connected()
obs = CustomObservable(2, ["XY", "ZZ", "ZI", "IY"])
qcnn_obs = qcnn.QCNNObservable(obs).get_operator(np.random.rand(obs.num_parameters))
qcnn_obs

SparsePauliOp(['IXIY', 'IZIZ', 'IZII', 'IIIY'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j])

In [14]:
qcnn_circuit = qcnn.get_circuit(np.random.rand(qcnn.num_features),np.random.rand(qcnn.num_parameters))
qcnn_circuit

In [15]:
measured_tree = OpTree.gen_expectation_tree(qcnn_circuit, qcnn_obs)

print(measured_tree)


     ┌─────────┐           ┌─────────┐┌─────────┐           ┌───────┐
q_0: ┤0        ├───────────┤1        ├┤0        ├───────────┤0      ├
     │  Conv_0 │┌─────────┐│         ││         │           │       │
q_1: ┤1        ├┤0        ├┤         ├┤1        ├───────────┤  FC_1 ├
     ├─────────┤│  Conv_0 ││  Conv_0 ││         │┌─────────┐│       │
q_2: ┤0        ├┤1        ├┤         ├┤  Pool_0 ├┤0        ├┤1      ├
     │  Conv_0 │└─────────┘│         ││         ││         │└───────┘
q_3: ┤1        ├───────────┤0        ├┤         ├┤1        ├─────────
     └─────────┘           └─────────┘│         ││  Pool_0 │         
c_0: ═════════════════════════════════╡0        ╞╡         ╞═════════
                                      └─────────┘│         │         
c_1: ════════════════════════════════════════════╡0        ╞═════════
                                                 └─────────┘         

 with observable 
SparsePauliOp(['IXIY', 'IZIZ', 'IZII', 'IIIY'],
              coeffs=[

C:\Users\DKR\AppData\Roaming\Python\Python310\site-packages\qiskit\circuit\quantumcircuit.py:1818: RuntimeWarning: Cregbundle set to False since an instruction needs to refer to individual classical wire
  return circuit_drawer(


In [29]:
from qiskit.primitives import Estimator,BackendEstimator
from qiskit import Aer
from squlearn import Executor

executor = Executor("qasm_simulator",shots=10000,primitive_seed=0)
estimator = executor.get_estimator()

OpTree.evaluate.evaluate_tree_with_estimator(measured_tree,{},estimator)

0.7034

In [30]:
from qiskit.primitives import BackendSampler
from qiskit import Aer

executor = Executor("qasm_simulator",shots=10000,primitive_seed=0)
sampler = executor.get_sampler()

OpTree.evaluate.evaluate_tree_with_sampler(measured_tree,{},sampler)

Traceback (most recent call last):
  File "C:\Users\DKR\Documents\Git Repositories\squlearn_fork\squlearn\src\squlearn\util\executor.py", line 554, in _primitive_run
    job = run()
  File "C:\Users\DKR\Documents\Git Repositories\squlearn_fork\squlearn\src\squlearn\util\executor.py", line 838, in run
    return self.sampler.run(circuits, parameter_values, **kwargs)
  File "C:\Users\DKR\AppData\Roaming\Python\Python310\site-packages\qiskit\primitives\base\base_sampler.py", line 136, in run
    circuits, parameter_values = validation._validate_sampler_args(circuits, parameter_values)
  File "C:\Users\DKR\AppData\Roaming\Python\Python310\site-packages\qiskit\primitives\base\validation.py", line 87, in _validate_sampler_args
    circuits = _validate_circuits(circuits, requires_measure=True)
  File "C:\Users\DKR\AppData\Roaming\Python\Python310\site-packages\qiskit\primitives\base\validation.py", line 123, in _validate_circuits
    raise ValueError(
ValueError: The 0-th circuit does not hav

RuntimeError: Failed to execute job on simulator!